In [12]:
%pip install spacy
%pip install nltk
!python -m spacy download en_core_web_sm  # SpaCy English model

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [19]:
#  Import all necessary libraries
import spacy
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [20]:
#  Download required NLTK resources (run once)
nltk.download('punkt')          # For tokenization
nltk.download('stopwords')      # For stopwords
nltk.download('wordnet')        # For lemmatization
nltk.download('vader_lexicon')  # For sentiment analysis

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Garvit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Garvit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Garvit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Garvit\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [14]:
# Load SpaCy model and initialize tools
nlp = spacy.load("en_core_web_sm")  # Load English model for NER and tokenization
sia = SentimentIntensityAnalyzer()  # Sentiment analyzer
stop_words = set(stopwords.words('english'))  # Stopwords list
lemmatizer = WordNetLemmatizer()  # Lemmatizer for normalizing words

In [15]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Check if entity ruler already exists
if "entity_ruler" not in nlp.pipe_names:
    ruler = nlp.add_pipe("entity_ruler", before="ner")
else:
    ruler = nlp.get_pipe("entity_ruler")

# Define patterns
patterns = [
    {"label": "CLIMATE", "pattern": "seas"},
    {"label": "CLIMATE", "pattern": "deforestation"},
    {"label": "CLIMATE", "pattern": "flood"},
    {"label": "CLIMATE", "pattern": "drought"},
    {"label": "URGENCY", "pattern": "threaten"},
    {"label": "URGENCY", "pattern": "killing"},
    {"label": "URGENCY", "pattern": "help"},
    {"label": "URGENCY", "pattern": "now"}
]

# Add patterns
ruler.add_patterns(patterns)


In [16]:

# Define location weights (higher for underrepresented regions)
location_weights = {
    "Tuvalu": 1.5,    # Small island nation, highly vulnerable
    "Kenya": 1.4,     # Developing nation, often overlooked
    "Brazil": 1.2,    # Large but with marginalized communities
    "USA": 0.8,       # Well-represented, lower weight
    "Europe": 0.9     # Broad region, often overrepresented
}

In [17]:
#  Function to process a single text
def analyze_climate_text(text):
    # Tokenize and clean with NLTK
    tokens = word_tokenize(text.lower())
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token.isalpha()]
    print(f"Cleaned Tokens: {cleaned_tokens}")
    
    # Process with SpaCy for NER and custom entities
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    locations = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
    climate_issues = [ent.text for ent in doc.ents if ent.label_ == "CLIMATE"]
    urgency_markers = [ent.text for ent in doc.ents if ent.label_ == "URGENCY"]
    print(f"Entities: {entities}")
    
    # Sentiment analysis with NLTK
    sentiment = sia.polarity_scores(text)
    print(f"Sentiment: {sentiment}")
    
    # Calculate urgency score
    location = locations[0] if locations else "Unknown"
    weight = location_weights.get(location, 1.0)  # Default weight 1.0
    urgency_score = (len(urgency_markers) * 0.3 + abs(sentiment["compound"])) * weight
    level = ("Urgent" if urgency_score > 0.5 else 
             "High Concern" if urgency_score > 0.3 else 
             "Notable")
    
    # Generate output
    issue_str = climate_issues[0] if climate_issues else "climate issue"
    urgency_str = f"{urgency_markers[0]}" if urgency_markers else "none"
    result = {
        "Location": location,
        "Level": level,
        "Issue": issue_str,
        "Sentiment": sentiment["compound"],
        "Urgency Marker": urgency_str,
        "Score": round(urgency_score, 2)
    }
    return result

In [18]:
#  Test the project with sample texts
sample_texts = [
    "Rising seas threaten Tuvalu!",              # Small island voice
    "Deforestation in Brazil is killing us.",    # Forest community voice
    "Drought in Kenya needs help now!",          # Arid region voice
    "Flood risks rise in Europe."                # Well-represented region
]

In [11]:
#  Run analysis and display results
print("Climate Crisis Voice Amplifier Results:")
print("-" * 50)
for text in sample_texts:
    print(f"\nAnalyzing: {text}")
    result = analyze_climate_text(text)
    print(f"Location: {result['Location']}")
    print(f"Priority: {result['Level']}")
    print(f"Issue: {result['Issue']}")
    print(f"Sentiment Score: {result['Sentiment']}")
    print(f"Urgency Marker: {result['Urgency Marker']}")
    print(f"Urgency Score: {result['Score']}")
    print("-" * 50)

Climate Crisis Voice Amplifier Results:
--------------------------------------------------

Analyzing: Rising seas threaten Tuvalu!
Cleaned Tokens: ['rising', 'sea', 'threaten', 'tuvalu']
Entities: [('seas', 'CLIMATE'), ('threaten', 'URGENCY')]
Sentiment: {'neg': 0.491, 'neu': 0.509, 'pos': 0.0, 'compound': -0.4389}
Location: Unknown
Priority: Urgent
Issue: seas
Sentiment Score: -0.4389
Urgency Marker: threaten
Urgency Score: 0.74
--------------------------------------------------

Analyzing: Deforestation in Brazil is killing us.
Cleaned Tokens: ['deforestation', 'brazil', 'killing', 'u']
Entities: [('Brazil', 'GPE'), ('killing', 'URGENCY')]
Sentiment: {'neg': 0.468, 'neu': 0.532, 'pos': 0.0, 'compound': -0.6597}
Location: Brazil
Priority: Urgent
Issue: climate issue
Sentiment Score: -0.6597
Urgency Marker: killing
Urgency Score: 1.15
--------------------------------------------------

Analyzing: Drought in Kenya needs help now!
Cleaned Tokens: ['drought', 'kenya', 'need', 'help']
Ent